# **SETUP**

## Import

In [1]:
from __future__ import division
import pickle
import numpy as np
import os
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

## Init

In [2]:
auc_results = []
recall_results = []

seed = 76424236
np.random.seed(seed=seed)

folder_name = f"./generated_data/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load training set

In [3]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['max_user'] = 7177
raw_data['max_item'] = 10729
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

## Define Model

In [4]:
import tensorflow as tf                     # Code to avoid tf using cached embeddings
tf.compat.v1.reset_default_graph()

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"KuaiRec",
                                     item_serving_size=500)
auc_evaluator = AUC()

cml_model.load(folder_name+"cml-KuaiRec")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2024-04-24 12:50:28.829024: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-04-24 12:50:28.832397: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 4192050000 Hz
2024-04-24 12:50:28.833157: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556c446ac1c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-24 12:50:28.833172: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


INFO:tensorflow:Restoring parameters from ./generated_data/cml-KuaiRec


# **DEFINE FUNCTION**

In [5]:
def eq(infilename, infilename_neg, trainfilename, gamma=1.0, K=30):

    # Load the pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    
    # Merge the two dictionaries
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    
    Zui = dict()
    Ni = dict()

    # Compute the frequency of each item in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Count the number of users with at least one positive item in the test set
    # (not the smartest way)
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break

    # Compute the recommendations
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    # Compute the scores using AUC and compute the recall
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [6]:
def aoa(infilename, infilename_neg, trainfilename, K=1):

    # Load pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    
    # Merge the two dictionaries
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
        
    Zui = dict()
    Ni = dict()
    
    # Count item frequencies in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    
    # Count the number of users with at least one positive item in the test set 
    # (not the smartest way)
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break

    # Compute the recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
            
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute the scores using AUC and compute the recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser]))
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0
            denominator += 1 
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

# **TEST**

## Generate Raw Results

In [7]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = folder_name+"cml-KuaiRec-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = folder_name+"cml-KuaiRec-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 7176/7176 [01:17<00:00, 92.40it/s]


{'AUC': [0.556010101010101,
  0.4992398648648649,
  0.5087919463087248,
  0.544421768707483,
  0.5247324414715718,
  0.5150841750841751,
  0.4965886287625418,
  0.48907407407407405,
  0.504679054054054,
  0.4912671232876712,
  0.5544107744107744,
  0.5182881355932205,
  0.5299496644295303,
  0.44734375000000004,
  0.4981939799331104,
  0.5011655405405404,
  0.5152173913043478,
  0.4899826989619377,
  0.4904347826086956,
  0.47169999999999995,
  0.4901677852348993,
  0.47605,
  0.49714765100671143,
  0.5306354515050168,
  0.4827333333333333,
  0.48431438127090304,
  0.5405872483221478,
  0.48506711409395975,
  0.5055442176870748,
  0.5033217993079585,
  0.4584459459459459,
  0.48657094594594597,
  0.5135102739726027,
  0.438758389261745,
  0.5206688963210703,
  0.5291864406779662,
  0.5186666666666667,
  0.4763166666666666,
  0.5052033898305085,
  0.4668896321070235,
  0.5358862876254181,
  0.49731418918918924,
  0.4961148648648647,
  0.5188793103448276,
  0.5227027027027027,
  0.548093

## Compute results

In [8]:
eq(folder_name+"cml-KuaiRec-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-KuaiRec-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5)

{'auc': 0.8519140061717886, 'recall': 0.5378264210310162}

In [9]:
eq(folder_name+"cml-KuaiRec-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-KuaiRec-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2)

{'auc': 0.8469662794391772, 'recall': 0.5216021101398857}

In [10]:
eq(folder_name+"cml-KuaiRec-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-KuaiRec-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3)

{'auc': 0.8406755337419534, 'recall': 0.4996077803018351}

In [11]:
aoa(folder_name+"cml-KuaiRec-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-KuaiRec-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy")

{'auc': 0.9148412723461649, 'recall': 0.11669449955279332}